In [6]:
import pandas as pd
import numpy as np
Billing = pd.read_csv("Billing.csv")
DiseaseCase = pd.read_csv("DiseaseCase.csv")
Lab = pd.read_csv("LabReduced.csv", dtype={"Name_calc": str, "CodeType_orig": str, "CodeType_calc": str, "Code_orig": str, "Code_calc": str})
Exam = pd.read_csv("Exam.csv")
Risk = pd.read_csv("RiskFactor.csv")
Patient = pd.read_csv("Patient.csv")
csvs = [Lab, Exam]

c:\users\susu1\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
for x in csvs :
    print(x.columns)

Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc',
       'TestResult_calc', 'UnitOfMeasure_calc'],
      dtype='object')
Index(['Exam_ID', 'Patient_ID', 'Encounter_ID', 'Exam1', 'Result1_calc',
       'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'PairingMethod',
       'DateCreated'],
      dtype='object')


In [3]:
# Compare # of unique DM2 patients in Billing and DiseaseCase
diabetes = ["Diabetes Mellitus (ml)","Diabetes Mellitus"]
DC_dm2 = DiseaseCase.loc[DiseaseCase['Disease'].isin(diabetes)]
DC_dm2_id = DC_dm2['Patient_ID'].unique()

Billing2 = Billing.dropna(subset=['DiagnosisCode_calc'])
Billing2 = Billing2[pd.to_numeric(Billing2.DiagnosisCode_calc,errors='coerce').notnull()]
Billing2['DiagnosisCode_calc'] = pd.to_numeric(Billing2['DiagnosisCode_calc'])
BL_dm2 = Billing2.loc[(Billing2['DiagnosisCode_calc'] >= 250.0) & (Billing2['DiagnosisCode_calc'] < 251.0)]
BL_dm2_id = BL_dm2['Patient_ID'].unique()

print(len(DC_dm2_id), len(BL_dm2_id)) 
print (DC_dm2_id)
# The unique list of DM2 patients_IDs are stored in the DC_dm2_id variable

149376 73359
[ 1000100523789  1000100525567  1000100552985 ... 12120090048604
  5050130005702  5050130006667]


In [72]:
#A1C Levels
import re

a1c = Lab.loc[Lab["Code_calc"] == '17856-6']
#print(a1c)

a1c['TestResult_calc'] = a1c['TestResult_calc'].str.replace('\W', '')

a1c['TestResult_calc'] = a1c['TestResult_calc'].astype(float)    

p1 = a1c.loc[a1c["TestResult_calc"] < 7]
p1['Risk_FactorA1C'] = '1'

p2 = a1c.loc[a1c["TestResult_calc"] == 7]
p2['Risk_FactorA1C'] = '2'

p3 = a1c.loc[a1c["TestResult_calc"] > 7]
p3['Risk_FactorA1C'] = '3'

dataFrames = [p1, p2, p3]
a1c = pd.concat(dataFrames)
a1c['TestResA1C'] = a1c['TestResult_calc']
#print(a1c)

 

<ipython-input-72-f10cea6a99b6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1c['TestResult_calc'] = a1c['TestResult_calc'].str.replace('\W', '')
<ipython-input-72-f10cea6a99b6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1c['TestResult_calc'] = a1c['TestResult_calc'].astype(float)
<ipython-input-72-f10cea6a99b6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

             Patient_ID PerformedDate               Name_calc Code_calc  \
525       1000100166799    2019-10-07  Hemoglobin A1c (HbA1c)   17856-6   
759       1000100034604    2007-04-09  Hemoglobin A1c (HbA1c)   17856-6   
929       1000100082686    2017-03-06  Hemoglobin A1c (HbA1c)   17856-6   
1266      1000100163884    2007-10-20  Hemoglobin A1c (HbA1c)   17856-6   
2637      1000100165347    2007-04-25  Hemoglobin A1c (HbA1c)   17856-6   
...                 ...           ...                     ...       ...   
25337443  4000000679820    2017-10-18  Hemoglobin A1c (HbA1c)   17856-6   
25337446  4000000679820    2018-03-22  Hemoglobin A1c (HbA1c)   17856-6   
25337451  4000000679820    2019-04-24  Hemoglobin A1c (HbA1c)   17856-6   
25337514  4000000679414           NaN  Hemoglobin A1c (HbA1c)   17856-6   
25337543  4000000680009    2018-06-15  Hemoglobin A1c (HbA1c)   17856-6   

          TestResult_calc UnitOfMeasure_calc Risk_FactorA1C  TestResA1C  
525                   6.0

In [74]:
# Blood Pressure
#Exam.head()

BP = Exam.loc[Exam["Exam1"] == 'sBP (mmHg)']

BP['Result1_calc'] = BP['Result1_calc'].replace('\W', '')

BP['Result1_calc'] = BP['Result1_calc'].astype(float)    

BP['Result2_calc'] = BP['Result1_calc'].replace('\W', '')

BP['Result2_calc'] = BP['Result1_calc'].astype(float) 

p1 = BP.loc[BP["Result1_calc"] < 120]
p1 = p1.loc[p1['Result2_calc'] < 70]
p1['Risk_FactorBP'] = '1'

p2 = BP.loc[BP["Result1_calc"] < 130]
p2 = p2.loc[p2["Result2_calc"] < 80]
p2['Risk_FactorBP'] = '2'

p3 = BP.loc[BP["Result1_calc"] >= 140]
p3 = p3.loc[p3["Result2_calc"] >= 90]
p3['Risk_FactorBP'] = '3'
    
dataFrames = [p1, p2, p3]
BP = pd.concat(dataFrames)
BP['sysBP'] = BP['Result1_calc']
BP['diaBP'] = BP['Result2_calc']
#print(BP)

<ipython-input-74-57da40cc453b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BP['Result1_calc'] = BP['Result1_calc'].replace('\W', '')
<ipython-input-74-57da40cc453b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BP['Result1_calc'] = BP['Result1_calc'].astype(float)
<ipython-input-74-57da40cc453b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

         Exam_ID    Patient_ID  Encounter_ID       Exam1  Result1_calc  \
13499     147768  4.000000e+12  4.060000e+13  sBP (mmHg)          66.0   
30645     149578  4.000000e+12  4.060000e+13  sBP (mmHg)          67.0   
49387     111460  4.000000e+12           NaN  sBP (mmHg)          63.0   
57503     154453  4.000000e+12  4.060000e+13  sBP (mmHg)          69.0   
66019     378713  4.000000e+12  4.066000e+13  sBP (mmHg)          63.0   
...          ...           ...           ...         ...           ...   
1048536  3041421  4.000000e+12           NaN  sBP (mmHg)         140.0   
1048543  3041539  4.000000e+12           NaN  sBP (mmHg)         140.0   
1048561  3239095  4.000000e+12  4.100000e+13  sBP (mmHg)         142.0   
1048562  3239207  4.000000e+12  4.100000e+13  sBP (mmHg)         162.0   
1048563  3239277  4.000000e+12  4.100000e+13  sBP (mmHg)         157.0   

              Exam2  Result2_calc UnitOfMeasure_calc PairingMethod  \
13499    dBP (mmHg)          66.0        

In [55]:
#Lipid Management

LDL = Lab.loc[Lab["Code_calc"] == '22748-8'] 
HDL = Lab.loc[Lab["Code_calc"] == '14646-4'] 
Tri = Lab.loc[Lab["Code_calc"] == '14927-8']

LDL['TestResult_calc'] = LDL['TestResult_calc'].str.replace('\W', '')
LDL['TestResult_calc'] = LDL['TestResult_calc'].astype(float)   

p1LDL = LDL.loc[LDL["TestResult_calc"] < 5.6 ]
p1LDL['Risk_FactorL'] = 1

p2LDL = LDL.loc[LDL["TestResult_calc"] < 7.2 ]
p2LDL['Risk_FactorL'] = 2

p3LDL = LDL.loc[LDL["TestResult_calc"] >= 7.2 ]
p3LDL['Risk_FactorL'] = 3

dataFrames = [p1LDL, p2LDL, p3LDL]
LDLRF = pd.concat(dataFrames)
LDLRF['TestRes_LDL'] = LDL['TestResult_calc']
#print(LDLRF)

HDL['TestResult_calc'] = HDL['TestResult_calc'].str.replace('\W', '')
HDL['TestResult_calc'] = HDL['TestResult_calc'].astype(float)   

p1HDL = HDL.loc[HDL["TestResult_calc"] > 3.3 ]
p1HDL['Risk_FactorH'] = 1

p2HDL = HDL.loc[HDL["TestResult_calc"] < 3.3 ]
p2HDL = p2HDL.loc[p2HDL["TestResult_calc"] > 2.2 ]
p2HDL['Risk_FactorH'] = 2

p3HDL = HDL.loc[HDL["TestResult_calc"] < 2.2 ]
p3HDL['Risk_FactorH'] = 3

dataFrames = [p1HDL, p2HDL, p3HDL]
HDLRF = pd.concat(dataFrames)
HDLRF['TestRes_HDL'] = HDL['TestResult_calc']
#print(HDLRF)

Tri['TestResult_calc'] = Tri['TestResult_calc'].str.replace('\W', '')
Tri['TestResult_calc'] = Tri['TestResult_calc'].astype(float)   

p1Tri = Tri.loc[Tri["TestResult_calc"] <= 1.7 ]
p1Tri['Risk_FactorT'] = 1

p2Tri = Tri.loc[Tri['TestResult_calc'] > 1.7]
p2Tri = p2Tri.loc[p2Tri["TestResult_calc"] < 5.6 ]
p2Tri['Risk_FactorT'] = 2

p3Tri = Tri.loc[Tri["TestResult_calc"] >= 5.7 ]
p3Tri['Risk_FactorT'] = 3

dataFrames = [p1Tri, p2Tri, p3Tri]
TriRF = pd.concat(dataFrames)
TriRF['TestRes_Tri'] = Tri['TestResult_calc']
#print(TriRF)



placeHolder = pd.concat([LDLRF, HDLRF,TriRF])
#new = placeHolder.loc[: , "Risk_FactorL":"Risk_FactorT"]
newnew = placeHolder[['Risk_FactorL', 'Risk_FactorH', 'Risk_FactorT']].copy()
#print(placeHolder)
#print(newnew)
newnew['RFMean'] = newnew.mean(axis=1)
#print(newnew)


placeHolder['LipidsRF'] = newnew['RFMean']
lipids = placeHolder.drop(['Code_calc', 'UnitOfMeasure_calc', 'TestResult_calc', 'Risk_FactorL', 'Risk_FactorH', 'Risk_FactorT'], axis =1)
#print(placeHolder)
#print(lipids)


##### FIX THIS ########
#new = pd.merge(LDLRF, HDLRF, TriRF, on=['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc']).set_index(['Patient_ID','PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc']).sum(axis=1)
#print(new)
#new = pd.concat((iDF.set_index('Patient_ID') for iDF in [LDLRF, HDLRF, TriRF]),axis=1, join='inner').reset_index()
#print(new)
#newRiskFactor = pd.merge(LDLRF, HDLRF, on=['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc']).set_index(['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc']).sum(axis=1)
#print(newRiskFactor)

<ipython-input-55-0c03c0ff3743>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LDL['TestResult_calc'] = LDL['TestResult_calc'].str.replace('\W', '')
<ipython-input-55-0c03c0ff3743>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LDL['TestResult_calc'] = LDL['TestResult_calc'].astype(float)
<ipython-input-55-0c03c0ff3743>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

             Patient_ID PerformedDate                  Name_calc Code_calc  \
240       1000100164439    2014-01-17  Cholesterol in LDL (LDLc)   22748-8   
3461      1000100164117    2007-07-21  Cholesterol in LDL (LDLc)   22748-8   
6993      1000100166700    2016-08-02  Cholesterol in LDL (LDLc)   22748-8   
7297      1000100080422    2011-04-04  Cholesterol in LDL (LDLc)   22748-8   
10551     1000100083781    2007-08-07  Cholesterol in LDL (LDLc)   22748-8   
...                 ...           ...                        ...       ...   
25337142  4000000668610    2017-11-03  Cholesterol in LDL (LDLc)   22748-8   
25337148  4000000668610    2018-10-16  Cholesterol in LDL (LDLc)   22748-8   
25337231  4000000668610    2018-06-05  Cholesterol in LDL (LDLc)   22748-8   
25337252  4000000679414    2019-07-17  Cholesterol in LDL (LDLc)   22748-8   
25337399  4000000679617           NaN  Cholesterol in LDL (LDLc)   22748-8   

          TestResult_calc UnitOfMeasure_calc  Risk_FactorL  Tes

<ipython-input-55-0c03c0ff3743>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HDL['TestResult_calc'] = HDL['TestResult_calc'].str.replace('\W', '')
<ipython-input-55-0c03c0ff3743>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HDL['TestResult_calc'] = HDL['TestResult_calc'].astype(float)
<ipython-input-55-0c03c0ff3743>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

             Patient_ID PerformedDate                  Name_calc Code_calc  \
35        1000100166298    2017-10-19  Cholesterol in HDL (HDLc)   14646-4   
86        1000100165244    2016-10-14  Cholesterol in HDL (HDLc)   14646-4   
143       1000100164719    2014-03-18  Cholesterol in HDL (HDLc)   14646-4   
166       1000100164439    2014-01-17  Cholesterol in HDL (HDLc)   14646-4   
337       1000100415873    2018-04-30  Cholesterol in HDL (HDLc)   14646-4   
...                 ...           ...                        ...       ...   
25324964  4000000666954    2017-06-29  Cholesterol in HDL (HDLc)   14646-4   
25326392  4000000679069    2018-09-15  Cholesterol in HDL (HDLc)   14646-4   
25332989  4000000674631    2019-05-30  Cholesterol in HDL (HDLc)   14646-4   
25334010  4000000676050    2016-06-13  Cholesterol in HDL (HDLc)   14646-4   
25334771  4000000679030    2019-06-05  Cholesterol in HDL (HDLc)   14646-4   

          TestResult_calc UnitOfMeasure_calc  Risk_FactorH  Tes

<ipython-input-55-0c03c0ff3743>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tri['TestResult_calc'] = Tri['TestResult_calc'].str.replace('\W', '')
<ipython-input-55-0c03c0ff3743>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tri['TestResult_calc'] = Tri['TestResult_calc'].astype(float)
<ipython-input-55-0c03c0ff3743>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

             Patient_ID PerformedDate           Name_calc Code_calc  \
10406     1000100082761    2005-02-04  Triglycerides (TG)   14927-8   
10417     1000100165076    2015-03-10  Triglycerides (TG)   14927-8   
21925     1000100081106    2012-03-06  Triglycerides (TG)   14927-8   
27925     1000100165406    2008-06-19  Triglycerides (TG)   14927-8   
46656     1000100250096    2016-03-17  Triglycerides (TG)   14927-8   
...                 ...           ...                 ...       ...   
25337232  4000000668610    2018-06-05  Triglycerides (TG)   14927-8   
25337302  4000000679414    2019-07-17  Triglycerides (TG)   14927-8   
25337335  4000000679617           NaN  Triglycerides (TG)   14927-8   
25337346  4000000668754    2019-07-23  Triglycerides (TG)   14927-8   
25337349  4000000668754    2019-10-17  Triglycerides (TG)   14927-8   

          TestResult_calc UnitOfMeasure_calc  Risk_FactorT  TestRes_Tri  
10406                 1.0             mmol/L             1          1.0  

In [75]:
#Nephropathy
#L1: <30 mg/g ( !!! 3.39 mg/mmol) albumin creatinine ratio
#L2 : 30-300 mg/g (!!! 3.39 - 33.9 mg/mmol)
#L3 : >300 mg/g ( !!! 33.9 mg/mmol)
alb = Lab.loc[Lab["Code_calc"] == '32294-1']
#print(alb)

alb['TestResult_calc'] = alb['TestResult_calc'].str.replace('\W', '')

alb['TestResult_calc'] = alb['TestResult_calc'].astype(float) 

p1 = alb.loc[alb["TestResult_calc"] <= 3.39 ]
p1['Risk_FactorALB'] = 1

p2 = alb.loc[alb['TestResult_calc'] > 3.39]
p2 = p2.loc[p2["TestResult_calc"] <= 33.9 ]
p2['Risk_FactorALB'] = 2

p3 = alb.loc[alb["TestResult_calc"] > 33.9 ]
p3['Risk_FactorALB'] = 3

dataFrames = [p1, p2, p3]
albRF = pd.concat(dataFrames)
albRF['AlbCreaRatio'] = albRF['TestResult_calc']
#print(albRF)

             Patient_ID PerformedDate                             Name_calc  \
30        1000100164439    2017-11-02  Urine Albumin/Creatinine Ratio (ACR)   
32        1000100164719    2017-10-16  Urine Albumin/Creatinine Ratio (ACR)   
115       1000100164439    2014-04-07  Urine Albumin/Creatinine Ratio (ACR)   
117       1000100164439    2014-03-07  Urine Albumin/Creatinine Ratio (ACR)   
119       1000100164439    2014-02-05  Urine Albumin/Creatinine Ratio (ACR)   
...                 ...           ...                                   ...   
25337005  4000000671176    2019-03-06  Urine Albumin/Creatinine Ratio (ACR)   
25337224  4000000668610    2016-12-15  Urine Albumin/Creatinine Ratio (ACR)   
25337402  4000000680009    2017-02-16  Urine Albumin/Creatinine Ratio (ACR)   
25337415  4000000680009    2017-08-23  Urine Albumin/Creatinine Ratio (ACR)   
25337529  4000000680009    2017-02-24  Urine Albumin/Creatinine Ratio (ACR)   

         Code_calc TestResult_calc UnitOfMeasure_ca

<ipython-input-75-844fd2499a7d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alb['TestResult_calc'] = alb['TestResult_calc'].str.replace('\W', '')
<ipython-input-75-844fd2499a7d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alb['TestResult_calc'] = alb['TestResult_calc'].astype(float)
<ipython-input-75-844fd2499a7d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

             Patient_ID PerformedDate                             Name_calc  \
218       1000100165244    2015-07-13  Urine Albumin/Creatinine Ratio (ACR)   
739       1000100034604    2006-11-21  Urine Albumin/Creatinine Ratio (ACR)   
1559      1000100163593    2007-09-02  Urine Albumin/Creatinine Ratio (ACR)   
2458      1000100086689    2014-07-08  Urine Albumin/Creatinine Ratio (ACR)   
2639      1000100165347    2007-04-25  Urine Albumin/Creatinine Ratio (ACR)   
...                 ...           ...                                   ...   
25336774  4000000679684           NaN  Urine Albumin/Creatinine Ratio (ACR)   
25336890  4000000667788    2017-02-01  Urine Albumin/Creatinine Ratio (ACR)   
25336979  4000000678793    2019-01-11  Urine Albumin/Creatinine Ratio (ACR)   
25337004  4000000671176    2019-03-04  Urine Albumin/Creatinine Ratio (ACR)   
25337005  4000000671176    2019-03-06  Urine Albumin/Creatinine Ratio (ACR)   

         Code_calc  TestResult_calc UnitOfMeasure_c

In [85]:
#Compute Final Risk Factor
df = [a1c, BP, lipids,albRF]
allRF = pd.concat(df)
#print(allRF)

justRF = allRF.drop(['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc'], axis =1)
justRF = justRF.drop(['Exam_ID', 'Encounter_ID', 'Exam1','Result1_calc', 'Exam2', 'Result2_calc','PairingMethod', 'DateCreated','TestRes_LDL','TestRes_HDL','TestRes_Tri'], axis = 1)
justRF = justRF.drop(['AlbCreaRatio', 'sysBP', 'diaBP', 'TestResA1C'], axis = 1)
#print(justRF)

#justRF['TotRF'] = justRF.sum(axis=1)
justRF = justRF.replace(np.nan,0)
justRF['TotRF'] = allRF[['Risk_FactorA1C','Risk_FactorBP','LipidsRF','Risk_FactorALB']].sum(axis=1)
#print(justRF)


         Risk_FactorA1C Risk_FactorBP  LipidsRF  Risk_FactorALB
525                   1           NaN       NaN             NaN
759                   1           NaN       NaN             NaN
929                   1           NaN       NaN             NaN
1266                  1           NaN       NaN             NaN
2637                  1           NaN       NaN             NaN
...                 ...           ...       ...             ...
25336774            NaN           NaN       NaN             3.0
25336890            NaN           NaN       NaN             3.0
25336979            NaN           NaN       NaN             3.0
25337004            NaN           NaN       NaN             3.0
25337005            NaN           NaN       NaN             3.0

[4822429 rows x 4 columns]
         Risk_FactorA1C Risk_FactorBP  LipidsRF  Risk_FactorALB  TotRF
525                   1             0       0.0             0.0    0.0
759                   1             0       0.0             0.

In [89]:
#final table

allInfo = allRF.copy()
#print(allInfo)

allInfo['RFtot'] = justRF['TotRF']
allInfo = allInfo.drop(['Name_calc', 'Code_calc', 'TestResult_calc', 'UnitOfMeasure_calc','Exam_ID', 'Encounter_ID', 'Exam1','Result1_calc', 'Exam2', 'Result2_calc','PairingMethod'], axis = 1)
allInfo = allInfo.drop(['Risk_FactorA1C','Risk_FactorBP','LipidsRF','Risk_FactorALB'], axis = 1)
print(allInfo)

            Patient_ID PerformedDate                             Name_calc  \
525       1.000100e+12    2019-10-07                Hemoglobin A1c (HbA1c)   
759       1.000100e+12    2007-04-09                Hemoglobin A1c (HbA1c)   
929       1.000100e+12    2017-03-06                Hemoglobin A1c (HbA1c)   
1266      1.000100e+12    2007-10-20                Hemoglobin A1c (HbA1c)   
2637      1.000100e+12    2007-04-25                Hemoglobin A1c (HbA1c)   
...                ...           ...                                   ...   
25336774  4.000001e+12           NaN  Urine Albumin/Creatinine Ratio (ACR)   
25336890  4.000001e+12    2017-02-01  Urine Albumin/Creatinine Ratio (ACR)   
25336979  4.000001e+12    2019-01-11  Urine Albumin/Creatinine Ratio (ACR)   
25337004  4.000001e+12    2019-03-04  Urine Albumin/Creatinine Ratio (ACR)   
25337005  4.000001e+12    2019-03-06  Urine Albumin/Creatinine Ratio (ACR)   

         Code_calc  TestResult_calc UnitOfMeasure_calc Risk_Fac